In [3]:
from pathlib import Path
BASE_DIR = Path("/Users/dsaluru/Desktop/GPU/bird-sql-forked/save_data/qwen2.5/")

GENERATION_DIR = BASE_DIR / "generations"
RESULTS_DIR = BASE_DIR / "with_results"
REWARD_DIR = BASE_DIR / "with_rewards"

### Generations

In [4]:
# load the generations jsonl file - data_gen.jsonl
import json

final_generations_data = {}
with open(GENERATION_DIR / "data_gen.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)

        if data['question_id'] not in final_generations_data:
            final_generations_data[data['question_id']] = {
                'question_id' : data['question_id'],
                'db_id' : data['db_id'],
                'question' : data['question'],
                'evidence' : data['evidence'],
                'SQL' : data['SQL'],
                'difficulty' : data['difficulty'],
                'responses' : []
            }
        curr_responses = []
        for item in data['responses']:
            curr_responses.append(data['responses'][item]['content'])
        final_generations_data[data['question_id']]['responses'] += curr_responses




In [5]:
mean_responses = 0
for key in final_generations_data:
    mean_responses += len(final_generations_data[key]['responses'])
mean_responses /= len(final_generations_data)
print(mean_responses)


154.206


In [6]:
import pandas as pd
pd.Series(final_generations_data[1473]['responses']).value_counts()

SELECT AVG(T1.Consumption) / 12 FROM yearmonth AS T1 INNER JOIN customers AS T2 ON T1.CustomerID = T2.CustomerID WHERE T2.Segment = 'SME' AND T1.Date BETWEEN 201301 AND 201312            15
SELECT AVG(T1.Consumption) / 12 FROM yearmonth AS T1 INNER JOIN customers AS T2 ON T1.CustomerID = T2.CustomerID WHERE T1.Date BETWEEN 201301 AND 201312 AND T2.Segment = 'SME'            14
SELECT AVG(T2.Consumption) / 12 FROM customers AS T1 INNER JOIN yearmonth AS T2 ON T1.CustomerID = T2.CustomerID WHERE T1.Segment = 'SME' AND T2.Date BETWEEN 201301 AND 201312            13
SELECT AVG(T1.Consumption) / 12 FROM yearmonth AS T1 INNER JOIN customers AS T2 ON T1.CustomerID = T2.CustomerID WHERE T2.Segment = 'SME' AND T1.Date BETWEEN '201301' AND '201312'         9
SELECT AVG(T1.Consumption) / 12 FROM yearmonth AS T1 INNER JOIN customers AS T2 ON T1.CustomerID = T2.CustomerID WHERE SUBSTR(T1.Date, 1, 4) = '2013' AND T2.Segment = 'SME'                7
                                                  

### With Results

In [7]:
final_results_data = {}
with open(RESULTS_DIR / "data_with_results.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)

        if data['question_id'] not in final_results_data:
            final_results_data[data['question_id']] = {
                'question_id' : data['question_id'],
                'db_id' : data['db_id'],
                'question' : data['question'],
                'evidence' : data['evidence'],
                'SQL' : data['SQL'],
                'difficulty' : data['difficulty'],
                'responses' : []
            }
        curr_responses = []
        for item in data['responses']:
            curr_responses.append((data['responses'][item]['content'], data['responses'][item]['evaluation']))
        final_results_data[data['question_id']]['responses'] += curr_responses

In [8]:
mean_responses = 0
for key in final_results_data:
    mean_responses += len(final_results_data[key]['responses'])
mean_responses /= len(final_results_data)
print(mean_responses)


83.298


In [9]:
import pandas as pd
mean_pass = 0
for key in final_results_data:
    try:
        mean_pass += pd.Series([x[1].split(':')[0] for x in final_results_data[key]['responses']]).value_counts()['pass']
    except:
        print("key : ", key)
mean_pass /= len(final_results_data)
print(mean_pass)

key :  532
key :  701
key :  704
key :  518
70.08


### with rewards

In [10]:
final_rewards_data = {}
with open(REWARD_DIR / "data_reward_all.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)

        if data['question_id'] not in final_rewards_data:
            final_rewards_data[data['question_id']] = {
                'question_id' : data['question_id'],
                'db_id' : data['db_id'],
                'question' : data['question'],
                'evidence' : data['evidence'],
                'SQL' : data['SQL'],
                'difficulty' : data['difficulty'],
                'responses' : []
            }
        curr_responses = []
        for item in data['responses']:
            if data['responses'][item]['evaluation'].split(':')[0] == 'pass' and data['responses'][item]['reward'] != None:
                curr_responses.append((data['responses'][item]['content'], data['responses'][item]['evaluation'], data['responses'][item]['reward']))
        final_rewards_data[data['question_id']]['responses'] += curr_responses

In [11]:
mean_responses = 0
for key in final_rewards_data:
    mean_responses += len(final_rewards_data[key]['responses'])
mean_responses /= len(final_rewards_data)
print(mean_responses)


63.998


In [12]:
# sort the responses by reward

for key in final_rewards_data:
    final_rewards_data[key]['responses'].sort(key=lambda x: x[2], reverse=True)


In [13]:

for k in range(1, 20):
    accuracy = 0
    for key in final_rewards_data:
        all_candidates_rewards_score = [x[2] for x in final_rewards_data[key]['responses']]
        all_candidates_rewards = [x[1] for x in final_rewards_data[key]['responses']]
        all_candidates_sqls = [x[0] for x in final_rewards_data[key]['responses']]
        top_k_candidates_rewards_score = all_candidates_rewards_score[:k]
        top_k_candidates_rewards = all_candidates_rewards[:k]
        top_k_candidates_sqls = all_candidates_sqls[:k]

        if len(final_rewards_data[key]['responses']) > 0:
            if 'pass: correct' in top_k_candidates_rewards:
                accuracy += 1
    accuracy /= len(final_rewards_data)
    print("accuracy for k = ", k, " is ", accuracy)


accuracy for k =  1  is  0.69
accuracy for k =  2  is  0.712
accuracy for k =  3  is  0.716
accuracy for k =  4  is  0.722
accuracy for k =  5  is  0.732
accuracy for k =  6  is  0.744
accuracy for k =  7  is  0.748
accuracy for k =  8  is  0.752
accuracy for k =  9  is  0.758
accuracy for k =  10  is  0.758
accuracy for k =  11  is  0.762
accuracy for k =  12  is  0.772
accuracy for k =  13  is  0.78
accuracy for k =  14  is  0.78
accuracy for k =  15  is  0.78
accuracy for k =  16  is  0.788
accuracy for k =  17  is  0.79
accuracy for k =  18  is  0.792
accuracy for k =  19  is  0.796


### Only California DB

In [14]:
final_rewards_data_cs = {key: final_rewards_data[key] for key in final_rewards_data if final_rewards_data[key]['db_id'] == 'california_schools'}

In [15]:

for k in range(1, 20):
    accuracy = 0
    for key in final_rewards_data_cs:
        all_candidates_rewards_score = [x[2] for x in final_rewards_data_cs[key]['responses']]
        all_candidates_rewards = [x[1] for x in final_rewards_data_cs[key]['responses']]
        all_candidates_sqls = [x[0] for x in final_rewards_data_cs[key]['responses']]
        top_k_candidates_rewards_score = all_candidates_rewards_score[:k]
        top_k_candidates_rewards = all_candidates_rewards[:k]
        top_k_candidates_sqls = all_candidates_sqls[:k]

        if len(final_rewards_data_cs[key]['responses']) > 0:
            if 'pass: incorrect' in top_k_candidates_rewards:
                accuracy += 1
    accuracy /= len(final_rewards_data_cs)
    print("accuracy for k = ", k, " is ", accuracy)


accuracy for k =  1  is  0.5333333333333333
accuracy for k =  2  is  0.6333333333333333
accuracy for k =  3  is  0.6666666666666666
accuracy for k =  4  is  0.6666666666666666
accuracy for k =  5  is  0.7
accuracy for k =  6  is  0.7
accuracy for k =  7  is  0.7
accuracy for k =  8  is  0.7
accuracy for k =  9  is  0.7
accuracy for k =  10  is  0.7
accuracy for k =  11  is  0.7
accuracy for k =  12  is  0.7
accuracy for k =  13  is  0.7
accuracy for k =  14  is  0.7333333333333333
accuracy for k =  15  is  0.7333333333333333
accuracy for k =  16  is  0.7333333333333333
accuracy for k =  17  is  0.7666666666666667
accuracy for k =  18  is  0.8
accuracy for k =  19  is  0.8


In [16]:
california_schools_db_schema = """
【DB_ID】 california_schools
【Schema】
# Table: frpm
[
(CDSCode:TEXT, Primary Key, Examples: [01100170109835, 01100170112607, 01100170118489]),
(Academic Year:TEXT, Examples: [2014-2015]),
(County Code:TEXT, Examples: [01, 02, 03]),
(District Code:INTEGER, Examples: [10017, 31609, 31617]),
(School Code:TEXT, Examples: [0109835, 0112607, 0118489]),
(County Name:TEXT, Examples: [Alameda, Alpine, Amador]),
(District Name:TEXT),
(School Name:TEXT, Examples: [FAME Public Charter]),
(District Type:TEXT, Examples: [County Office of Education (COE)]),
(School Type:TEXT, Examples: [K-12 Schools (Public)]),
(Educational Option Type:TEXT, Examples: [Traditional]),
(NSLP Provision Status:TEXT, Examples: [Breakfast Provision 2]),
(Charter School (Y/N):INTEGER, Examples: [1, 0]),
(Charter School Number:TEXT, Examples: [0728, 0811, 1049]),
(Charter Funding Type:TEXT, Examples: [Directly funded]),
(IRC:INTEGER, Examples: [1, 0]),
(Low Grade:TEXT, Examples: [K, 9, 1]),
(High Grade:TEXT, Examples: [12, 8, 5]),
(Enrollment (K-12):REAL, Examples: [1087.0, 395.0, 244.0]),
(Free Meal Count (K-12):REAL, Examples: [565.0, 186.0, 134.0]),
(Percent (%) Eligible Free (K-12):REAL, Examples: [0.519779208831647, 0.470886075949367, 0.549180327868853]),
(FRPM Count (K-12):REAL, Examples: [715.0, 186.0, 175.0]),
(Percent (%) Eligible FRPM (K-12):REAL, Examples: [0.657773689052438, 0.470886075949367, 0.717213114754098]),
(Enrollment (Ages 5-17):REAL, Examples: [1070.0, 376.0, 230.0]),
(Free Meal Count (Ages 5-17):REAL, Examples: [553.0, 182.0, 128.0]),
(Percent (%) Eligible Free (Ages 5-17):REAL, Examples: [0.516822429906542, 0.484042553191489, 0.556521739130435]),
(FRPM Count (Ages 5-17):REAL, Examples: [702.0, 182.0, 168.0]),
(Percent (%) Eligible FRPM (Ages 5-17):REAL, Examples: [0.65607476635514, 0.484042553191489, 0.730434782608696]),
(2013-14 CALPADS Fall 1 Certification Status:INTEGER, Examples: [1])
]
# Table: schools
[
(CDSCode:TEXT, Primary Key, Examples: [01100170000000, 01100170109835, 01100170112607]),
(NCESDist:TEXT, Examples: [0691051, 0600002, 0600003]),
(NCESSchool:TEXT, Examples: [10546, 10947, 12283]),
(StatusType:TEXT, Examples: [Active, Closed, Merged]),
(County:TEXT, Examples: [Alameda, Alpine, Amador]),
(District:TEXT),
(School:TEXT, Examples: [FAME Public Charter]),
(Street:TEXT, Examples: [313 West Winton Avenue]),
(StreetAbr:TEXT, Examples: [313 West Winton Ave.]),
(City:TEXT, Examples: [Hayward, Newark, Oakland]),
(Zip:TEXT, Examples: [94544-1136, 94560-5359, 94612-3355]),
(State:TEXT, Examples: [CA]),
(MailStreet:TEXT, Examples: [313 West Winton Avenue]),
(MailStrAbr:TEXT, Examples: [313 West Winton Ave.]),
(MailCity:TEXT, Examples: [Hayward, Newark, Oakland]),
(MailZip:TEXT, Examples: [94544-1136, 94560-5359, 94612]),
(MailState:TEXT, Examples: [CA]),
(Phone:TEXT, Examples: [(510) 887-0152, (510) 596-8901, (510) 686-4131]),
(Ext:TEXT, Examples: [130, 1240, 1200]),
(Website:TEXT, Examples: [www.acoe.org]),
(OpenDate:DATE, Examples: [2005-08-29]),
(ClosedDate:DATE, Examples: [2015-07-31]),
(Charter:INTEGER, Examples: [1, 0]),
(CharterNum:TEXT, Examples: [0728, 0811, 1049]),
(FundingType:TEXT, Examples: [Directly funded]),
(DOC:TEXT, Examples: [00, 31, 34]),
(DOCType:TEXT, Examples: [County Office of Education (COE)]),
(SOC:TEXT, Examples: [65, 66, 60]),
(SOCType:TEXT, Examples: [K-12 Schools (Public)]),
(EdOpsCode:TEXT, Examples: [TRAD, JUV, COMM]),
(EdOpsName:TEXT, Examples: [Traditional]),
(EILCode:TEXT, Examples: [ELEMHIGH, HS, ELEM]),
(EILName:TEXT, Examples: [Elementary-High Combination]),
(GSoffered:TEXT, Examples: [K-12, 9-12, K-8]),
(GSserved:TEXT, Examples: [K-12, 9-12, K-7]),
(Virtual:TEXT, Examples: [P, N, F]),
(Magnet:INTEGER, Examples: [0, 1]),
(Latitude:REAL, Examples: [37.658212, 37.521436, 37.80452]),
(Longitude:REAL, Examples: [-122.09713, -121.99391, -122.26815]),
(AdmFName1:TEXT, Examples: [L Karen, Laura, Clifford]),
(AdmLName1:TEXT, Examples: [Monroe, Robell, Thompson]),
(AdmEmail1:TEXT),
(AdmFName2:TEXT, Examples: [Sau-Lim (Lance), Jennifer, Annalisa]),
(AdmLName2:TEXT, Examples: [Tsang, Koelling, Moore]),
(AdmEmail2:TEXT),
(AdmFName3:TEXT, Examples: [Drew, Irma, Vickie]),
(AdmLName3:TEXT, Examples: [Sarratore, Munoz, Chang]),
(AdmEmail3:TEXT),
(LastUpdate:DATE, Examples: [2015-06-23])
]
# Table: satscores
[
(cds:TEXT, Primary Key, Examples: [10101080000000, 10101080109991, 10101080111682]),
(rtype:TEXT, Examples: [D, S]),
(sname:TEXT, Examples: [FAME Public Charter]),
(dname:TEXT, Examples: [Alameda County Office of Education]),
(cname:TEXT, Examples: [Alameda, Amador, Butte]),
(enroll12:INTEGER, Examples: [398, 62, 75]),
(NumTstTakr:INTEGER, Examples: [88, 17, 71]),
(AvgScrRead:INTEGER, Examples: [418, 503, 397]),
(AvgScrMath:INTEGER, Examples: [418, 546, 387]),
(AvgScrWrite:INTEGER, Examples: [417, 505, 395]),
(NumGE1500:INTEGER, Examples: [14, 9, 5])
]
【Foreign keys】
frpm.CDSCode=schools.CDSCode
satscores.cds=schools.CDSCode
"""

In [ ]:
import asyncio
import json
from openai import OpenAI, AsyncOpenAI
from pydantic import BaseModel
from typing import List
import anthropic
import os

# Initialize the synchronous and asynchronous clients
client = OpenAI(timeout=60.)
aclient = AsyncOpenAI(timeout=60.0)
anthropic_client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
# Use the asynchronous client for Anthropic as well
anthropic_async_client = anthropic.AsyncAnthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))


# Pydantic models (unchanged)
class ColumnConnection(BaseModel):
    input_words: str
    related_column: str
    rationale: str

class SQLResponse(BaseModel):
    cols_io_connection: List[ColumnConnection]
    full_sql_query: str


async def oai_call_async(system_prompt, user_prompt, model, temp=0.001, n=1, backup_model='gpt-4o-2024-08-06', effort='low', idx=0):
    """
    An asynchronous version of oai_call that uses aclient for non-blocking API calls.
    """
    raw_response_json = None
    try:
        if model in ['gpt-3.5-turbo', 'gpt-4-turbo', 'gpt-4o-2024-08-06', 'gpt-4o', 'gpt-4o-mini', 'gpt-4.1', 'gpt-4.1-mini', 'gpt-4.1-nano']:
            response = await aclient.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=temp,
                n=n,
                response_format={"type": "json_object"},
                seed=1234
            )
            try:
                raw_response_json = response.model_dump_json(indent=2)
            except:
                raw_response_json = str(response)
            if n == 1:
                out = json.loads(response.choices[0].message.content)
            else:
                out = [json.loads(response.choices[i].message.content) for i in range(len(response.choices))]

        elif model in ['gpt-5', 'gpt-5-mini', 'gpt-5-nano', 'o3', 'o3-mini', 'o4-mini']:
            # Assuming client.responses.parse is a synchronous call, we run it in a separate thread
            response = await asyncio.to_thread(
                client.responses.parse,
                model=model,
                input=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                text={"format": {"type": "json_object"}},
                reasoning={"effort": effort, 'summary': 'detailed'}
            )
            try:
                raw_response_json = response.model_dump_json(indent=2)
            except:
                raw_response_json = str(response)
            out = json.loads(response.output[-1].content[0].text)
            if n > 1:
                out = [out] * n

        elif model in ['claude-sonnet-4-20250514', 'claude-opus-4-20250514']:
            thinking_budgets = {'low': 1024, 'high': 4096}
            params = {
                "model": model,
                "max_tokens": 8000,
                "system": system_prompt,
                "messages": [{"role": "user", "content": [{"type": "text", "text": user_prompt}]}]
            }
            if effort in thinking_budgets:
                params["thinking"] = {"type": "enabled", "budget_tokens": thinking_budgets[effort]}

            response = await anthropic_async_client.messages.create(**params)
            try:
                raw_response_json = response.model_dump_json(indent=2)
            except:
                raw_response_json = str(response)
                
            # Assuming client.responses.parse is a synchronous call, we run it in a separate thread
            response_data = await asyncio.to_thread(
                client.responses.parse,
                model="gpt-4o",
                input=[
                    {"role": "system", "content": "Extract the json object from the following response. There should be only one json object in the response."},
                    {"role": "user", "content": response.content[-1].text}
                ],
                text={"format": {"type": "json_object"}}
            )
            out = json.loads(response_data.output[-1].content[0].text)
        else:
            response = await aclient.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=temp,
                n=n,
            )
            try:
                raw_response_json = response.model_dump_json(indent=2)
            except:
                raw_response_json = str(response)
            if n == 1:
                out = response.choices[0].message.content
            else:
                out = [response.choices[i].message.content for i in range(len(response.choices))]

    except Exception as e:
        print(e)
        print(f"\t{idx}: Switching to {backup_model}", flush=True)
        response = await aclient.chat.completions.create(
            model=backup_model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=temp,
            n=n,
            response_format={"type": "json_object"},
            seed=1234
        )
        try:
            raw_response_json = response.model_dump_json(indent=2)
        except:
            raw_response_json = str(response)
        if n == 1:
            out = json.loads(response.choices[0].message.content)
        else:
            out = [json.loads(response.choices[i].message.content) for i in range(len(response.choices))]

    return out, raw_response_json

In [18]:
import json
import sqlite3
import pandas as pd
import time
from tqdm import tqdm
import traceback

def query_database(db_path, query):
    time_limit_ms = 10000

    def progress_handler():
        current_time = time.perf_counter()
        ptime = (start_time + time_limit_ms / 1000) - current_time
#         print(f"Timer: {ptime}", flush=True)
        if current_time > (start_time + time_limit_ms / 1000):
            return 1  
        return 0

    try:
        conn = sqlite3.connect(db_path)
        conn.set_progress_handler(progress_handler, 100000000)
        start_time = time.perf_counter()        
        cursor = conn.cursor()
        cursor.execute(query)        
        rows = cursor.fetchall()
        df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
        conn.close()
        return True, df

    except Exception as e:
        return False, str(e)

In [19]:
cs_db_path = "/Users/dsaluru/Desktop/GPU/bird-sql-forked/data/test_databases/california_schools/california_schools.sqlite"


In [20]:
for key in final_rewards_data_cs:
    data = final_rewards_data_cs[key]
    question = data['question']
    hint = data['evidence']
    candidate_sqls_output = [x[1] for x in data['responses'][:5]]
    if data['difficulty'] in ['moderate']:
        if 'pass: correct' != candidate_sqls_output[0]:
            print(key, data['difficulty'], question, hint, data['SQL'])

24 moderate Give the names of the schools with the percent eligible for free meals in K-12 is more than 0.1 and test takers whose test score is greater than or equal to 1500? Percent eligible for free meals = Free Meal Count (K-12) / Total (Enrollment (K-12) SELECT T2.`School Name` FROM satscores AS T1 INNER JOIN frpm AS T2 ON T1.cds = T2.CDSCode WHERE CAST(T2.`Free Meal Count (K-12)` AS REAL) / T2.`Enrollment (K-12)` > 0.1 AND T1.NumGE1500 > 0
25 moderate Name schools in Riverside which the average of average math score for SAT is grater than 400, what is the funding type of these schools? Average of average math = sum(average math scores) / count(schools). SELECT T1.sname, T2.`Charter Funding Type` FROM satscores AS T1 INNER JOIN frpm AS T2 ON T1.cds = T2.CDSCode WHERE T2.`District Name` LIKE 'Riverside%' GROUP BY T1.sname, T2.`Charter Funding Type` HAVING CAST(SUM(T1.AvgScrMath) AS REAL) / COUNT(T1.cds) > 400
26 moderate State the names and full communication address of high schools

In [21]:
for key in final_rewards_data:
    data = final_rewards_data[key]
    if 'at most' in data['evidence'].lower():
        print(key)

36


In [38]:
curr_index = 85

In [39]:
print("Question: ", final_rewards_data_cs[curr_index]['question'])
print("Evidence: ", final_rewards_data_cs[curr_index]['evidence'])
print("Golden SQL: ", final_rewards_data_cs[curr_index]['SQL'])
print("Gen SQL: ", final_rewards_data_cs[curr_index]['responses'][0][0])

Question:  What is the Percent (%) Eligible Free (K-12) in the school administered by an administrator whose first name is Alusine. List the district code of the school.
Evidence:  Percent (%) Eligible Free (K-12) = `Free Meal Count (K-12)` / `Enrollment (K-12)` * 100%
Golden SQL:  SELECT T1.`Free Meal Count (K-12)` * 100 / T1.`Enrollment (K-12)`, T1.`District Code` FROM frpm AS T1 INNER JOIN schools AS T2 ON T1.CDSCode = T2.CDSCode WHERE T2.AdmFName1 = 'Alusine'
Gen SQL:  SELECT T2."Percent (%) Eligible Free (K-12)", T2."District Code" FROM schools AS T1 INNER JOIN frpm AS T2 ON T1.CDSCode = T2.CDSCode WHERE T1.AdmFName1 = 'Alusine'


In [40]:
query_database(cs_db_path, final_rewards_data_cs[curr_index]['SQL'])

(True,
    T1.`Free Meal Count (K-12)` * 100 / T1.`Enrollment (K-12)`  District Code
 0                                          70.151134                   64857)

In [41]:
query_database(cs_db_path, final_rewards_data_cs[curr_index]['responses'][0][0])

(True,
    Percent (%) Eligible Free (K-12)  District Code
 0                          0.701511          64857)

In [46]:
query = """SELECT T2.`Free Meal Count (K-12)` * 100 / T2.`Enrollment (K-12)`, T2."District Code" FROM schools AS T1 INNER JOIN frpm AS T2 ON T1.CDSCode = T2.CDSCode WHERE T1.AdmFName1 = 'Alusine'"""

In [47]:
query_database(cs_db_path, query)

(True,
    T2.`Free Meal Count (K-12)` * 100 / T2.`Enrollment (K-12)`  District Code
 0                                          70.151134                   64857)

In [ ]:
SELECT CharterNum, AvgScrWrite, RANK() OVER (ORDER BY AvgScrWrite DESC) AS WritingScoreRank FROM schools AS T1  INNER JOIN satscores AS T2 ON T1.CDSCode = T2.cds WHERE T2.AvgScrWrite > 499 AND CharterNum is not null

In [234]:
final_rewards_data_cs[17]['responses'][0][0]

'SELECT T2.CharterNum FROM satscores AS T1 INNER JOIN schools AS T2 ON T1.cds = T2.CDSCode WHERE T1.AvgScrWrite > 499 AND T2.CharterNum IS NOT NULL ORDER BY T1.AvgScrWrite DESC'

In [226]:
moderate_hard

8

In [228]:
print(california_schools_db_schema)


【DB_ID】 california_schools
【Schema】
# Table: frpm
[
(CDSCode:TEXT, Primary Key, Examples: [01100170109835, 01100170112607, 01100170118489]),
(Academic Year:TEXT, Examples: [2014-2015]),
(County Code:TEXT, Examples: [01, 02, 03]),
(District Code:INTEGER, Examples: [10017, 31609, 31617]),
(School Code:TEXT, Examples: [0109835, 0112607, 0118489]),
(County Name:TEXT, Examples: [Alameda, Alpine, Amador]),
(District Name:TEXT),
(School Name:TEXT, Examples: [FAME Public Charter]),
(District Type:TEXT, Examples: [County Office of Education (COE)]),
(School Type:TEXT, Examples: [K-12 Schools (Public)]),
(Educational Option Type:TEXT, Examples: [Traditional]),
(NSLP Provision Status:TEXT, Examples: [Breakfast Provision 2]),
(Charter School (Y/N):INTEGER, Examples: [1, 0]),
(Charter School Number:TEXT, Examples: [0728, 0811, 1049]),
(Charter Funding Type:TEXT, Examples: [Directly funded]),
(IRC:INTEGER, Examples: [1, 0]),
(Low Grade:TEXT, Examples: [K, 9, 1]),
(High Grade:TEXT, Examples: [12, 8,